In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

In [2]:
#from source.utils import *
from source.preprocess import *
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm

In [3]:
path = "../../Desktop/DLC_social_1/"

# Set up and design the project

In [4]:
with open('{}DLC_social_1_exp_conditions.pickle'.format(path), 'rb') as handle:
    Treatment_dict = pickle.load(handle)

In [5]:
#Which angles to compute?
bp_dict = {'B_Nose':['B_Left_ear','B_Right_ear'],
          'B_Left_ear':['B_Nose','B_Right_ear','B_Center','B_Left_flank'],
          'B_Right_ear':['B_Nose','B_Left_ear','B_Center','B_Right_flank'],
          'B_Center':['B_Left_ear','B_Right_ear','B_Left_flank','B_Right_flank','B_Tail_base'],
          'B_Left_flank':['B_Left_ear','B_Center','B_Tail_base'],
          'B_Right_flank':['B_Right_ear','B_Center','B_Tail_base'],
          'B_Tail_base':['B_Center','B_Left_flank','B_Right_flank']}

In [6]:
%%time
DLC_social_1 = project(path=path,#Path where to find the required files
                   smooth_alpha=0.85,                    #Alpha value for exponentially weighted smoothing
                   distances=['B_Center','B_Nose','B_Left_ear','B_Right_ear','B_Left_flank',
                              'B_Right_flank','B_Tail_base'],
                   ego=False,
                   angles=True,
                   connectivity=bp_dict,
                   arena='circular',                  #Type of arena used in the experiments
                   arena_dims=[380],                  #Dimensions of the arena. Just one if it's circular
                   video_format='.mp4',
                   table_format='.h5',
                   exp_conditions=Treatment_dict)

CPU times: user 2.58 s, sys: 786 ms, total: 3.37 s
Wall time: 1.13 s


# Run project

In [7]:
%%time
DLC_social_1_coords = DLC_social_1.run(verbose=True)
print(DLC_social_1_coords)
type(DLC_social_1_coords)

Loading trajectories...
Smoothing trajectories...
Computing distances...
Computing angles...
Done!
Coordinates of 47 videos across 4 conditions
CPU times: user 8.95 s, sys: 574 ms, total: 9.52 s
Wall time: 9.84 s


source.preprocess.coordinates

# Generate coords

In [8]:
%%time
ptest = DLC_social_1_coords.get_coords(center=True, polar=False, speed=0, length='00:10:00')
ptest._type

CPU times: user 800 ms, sys: 68.6 ms, total: 868 ms
Wall time: 825 ms


'coords'

In [9]:
%%time
dtest = DLC_social_1_coords.get_distances(speed=0, length='00:10:00')
dtest._type

CPU times: user 532 ms, sys: 391 ms, total: 923 ms
Wall time: 923 ms


'dists'

In [10]:
%%time
atest = DLC_social_1_coords.get_angles(degrees=True, speed=0, length='00:10:00')
atest._type

CPU times: user 131 ms, sys: 77.5 ms, total: 208 ms
Wall time: 207 ms


'angles'

# Visualization playground

In [11]:
#ptest.plot_heatmaps(['B_Center', 'W_Center'], i=1)

In [12]:
#Plot animation of trajectory over time with different smoothings
#plt.plot(ptest['Day2Test13DLC']['B_Center'].iloc[:5000]['x'],
#         ptest['Day2Test13DLC']['B_Center'].iloc[:5000]['y'], label='alpha=0.85')

#plt.xlabel('x')
#plt.ylabel('y')
#plt.title('Mouse Center Trajectory using different exponential smoothings')
#plt.legend()
#plt.show()

# Dimensionality reduction playground

In [13]:
#pca = ptest.pca(4, 1000)

In [14]:
#plt.scatter(*pca[0].T)
#plt.show()

# Preprocessing playground

In [15]:
mtest = merge_tables(DLC_social_1_coords.get_coords(center=True, polar=True, length='00:10:00'))#,
#                      DLC_social_1_coords.get_distances(speed=0, length='00:10:00'),
#                      DLC_social_1_coords.get_angles(degrees=True, speed=0, length='00:10:00'))

In [16]:
#pptest = mtest.preprocess(window_size=51, filter='gaussian', sigma=10, shift=20)

In [17]:
pttest = mtest.preprocess(window_size=11, window_step=6, filter=None, standard_scaler=True)
pttest.shape

(117507, 11, 28)

In [18]:
#plt.plot(pttest[2,:,2], label='normal')
#plt.plot(pptest[2,:,2], label='gaussian')
#plt.legend()
#plt.show()

# Trained models playground

### Seq 2 seq Variational Auto Encoder

In [19]:
from datetime import datetime
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.constraints import UnitNorm
from tensorflow.keras.layers import Bidirectional, Dense, Dropout
from tensorflow.keras.layers import Lambda, LSTM
from tensorflow.keras.layers import RepeatVector, TimeDistributed
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from source.model_utils import *
import keras as k
import tensorflow as tf

Using TensorFlow backend.


In [20]:
NAME = 'Baseline_VAE_short_512_10=warmup_begin'
log_dir = os.path.abspath(
    "logs/fit/{}_{}".format(NAME, datetime.now().strftime("%Y%m%d-%H%M%S"))
)
tensorboard_callback = k.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [21]:
from source.models import SEQ_2_SEQ_AE, SEQ_2_SEQ_VAE, SEQ_2_SEQ_VAEP

In [22]:
encoder, decoder, ae = SEQ_2_SEQ_AE(pttest.shape).build()

In [23]:
def kl_rate(y_true, y_pred):
    return 0

def mmd_rate(y_true, y_pred):
    return 0

In [36]:
encoder, generator, vae, kl_warmup_callback, mmd_warmup_callback = SEQ_2_SEQ_VAE(pttest.shape,
                                                                   loss='ELBO+MMD',
                                                                   kl_warmup_epochs=10,
                                                                   mmd_warmup_epochs=10).build()

In [42]:
encoder, generator, vaep, kl_warmup_callback, mmd_warmup_callback = SEQ_2_SEQ_VAEP(pttest.shape,
                                                                    loss='ELBO+MMD',
                                                                    kl_warmup_epochs=10,
                                                                    mmd_warmup_epochs=10).build()

In [26]:
#ae.summary()
#vae.summary()
#vaep.summary()

In [27]:
#from tensorflow.keras.utils import plot_model
#plot_model(vaep, show_shapes=True)

In [28]:
#plot_model(vae)

In [29]:
#np.random.shuffle(pttest)
pttrain = pttest[:-15000]
pttest  = pttest[-15000:]
pttrain = pttrain[:15000]

In [30]:
#lr_schedule = tf.keras.callbacks.LearningRateScheduler(
#    lambda epoch: 1e-3 * 10**(epoch / 20))

In [38]:
#tf.config.experimental_run_functions_eagerly(False)
history = vae.fit(x=pttrain[:-1], y=pttrain[:-1], epochs=100, batch_size=512, verbose=1,
                  validation_data=(pttest[:-1], pttest[:-1]),
                  callbacks=[tensorboard_callback, kl_warmup_callback, mmd_warmup_callback])

Train on 14999 samples, validate on 14999 samples
14999/14999 [==============================] - 45s 3ms/sample - loss: 866955.0147 - mae: 0.5701 - kl_rate: -0.0154 - mmd_rate: -0.0157 - val_loss: 1588031.0797 - val_mae: 0.8311 - val_kl_rate: -0.0327 - val_mmd_rate: -0.0333


In [44]:
#tf.config.experimental_run_functions_eagerly(False)
#history = vaep.fit(x=pttrain[:-1], y=[pttrain[:-1],pttrain[1:]], epochs=100, batch_size=512, verbose=1,
#                   validation_data=(pttest[:-1], [pttest[:-1],pttest[1:]]),
#                   callbacks=[tensorboard_callback, kl_warmup_callback, mmd_warmup_callback])

Train on 14999 samples, validate on 14999 samples
Epoch 1/100
14999/14999 [==============================] - 66s 4ms/sample - loss: 2664896.8373 - vaep_reconstruction_loss: 1282800.0000 - vaep_prediction_loss: 1331754.7500 - vaep_reconstruction_mae: 0.6994 - vaep_prediction_mae: 0.7126 - kl_rate: -0.0149 - mmd_rate: -0.0148 - val_loss: 3214011.2341 - val_vaep_reconstruction_loss: 1584510.1250 - val_vaep_prediction_loss: 1576251.8750 - val_vaep_reconstruction_mae: 0.8477 - val_vaep_prediction_mae: 0.8269 - val_kl_rate: -0.0316 - val_mmd_rate: -0.0312
Epoch 2/100
14999/14999 [==============================] - 60s 4ms/sample - loss: 1145254.4113 - vaep_reconstruction_loss: 534650.3750 - vaep_prediction_loss: 589985.5625 - vaep_reconstruction_mae: 0.4546 - vaep_prediction_mae: 0.4738 - kl_rate: 0.0829 - mmd_rate: 0.0844 - val_loss: 3047568.4644 - val_vaep_reconstruction_loss: 1506117.6250 - val_vaep_prediction_loss: 1492132.8750 - val_vaep_reconstruction_mae: 0.8216 - val_vaep_prediction_m

KeyboardInterrupt: 